# Tokenization

**概述（By AI）：**

*   **阐述 BPE 算法原理与动机：**
    *   **工作：** 分析了 Tokenization 面临的挑战（词表大小 vs. 未登录词），探讨了直接使用字节流的局限性，并阐述了结合统计信息（如字节对频率）进行分词的必要性。
    *   **结论：** 需要一种兼顾“表达能力”和“信息压缩率”的方法，BPE 通过利用统计信息，合并高频字节对，构建词表，可以在保持合理序列长度的同时，使 Token 具备一定的语义信息，适应 LLM 的需求。

*   **实现 BasicTokenizer：**
    *   **工作：** 编写了一个基础版本的 BPE Tokenizer (`BasicTokenizer`)，严格按照 BPE 算法逻辑，在每次合并后重新计算字节对频率。
    *   **结论：** 该实现方式逻辑清晰，易于理解 BPE 核心思想，但效率较低，因为涉及大量重复的计算和查找。

*   **实现 FasterTokenizer：**
    *   **工作：** 针对 `BasicTokenizer` 的效率问题，实现了一个优化版本 (`FasterTokenizer`)，改进了最长匹配查找（使用哈希表和最大长度限制）和合并过程（更新现有 token 列表而非完全重建）。
    *   **结论：** `FasterTokenizer` 在训练速度上相较于 `BasicTokenizer` 有显著提升（几十倍），尤其在处理更大语料时优势更明显。

*   **性能与准确性测试：**
    *   **工作：** 在不同大小的语料上测试并对比了 `BasicTokenizer` 和 `FasterTokenizer` 的训练、编码、解码时间及准确性。
    *   **结论：**
        *   `FasterTokenizer` 的训练加速比随语料规模增大而增加，符合预期。
        *   两种实现都能准确地编码和解码文本。
        *   由于 BPE 合并过程的贪心性质和可能的频率并列情况，不同实现（甚至同一实现在不同语料子集上）可能产生不同的词表和最终 token 序列。

*   **实现词表存取与Token可视化：**
    *   **工作：** 编写了 `save_vocab` 和 `load_vocab` 函数，使用 Base64 编码将 `bytes` 类型的词表保存到文件及从文件加载。同时，编写了 `visualize_tokens` 函数，使用 `termcolor` 库以随机颜色直观展示分词结果。
    *   **结论：** 这些辅助功能增强了 Tokenizer 的可用性和结果的可解释性。

*   **与 Tiktoken 对比分析：**
    *   **工作：** 使用 `final_tokenizer`（基于 `FasterTokenizer` 训练得到）和 OpenAI 的 `tiktoken` (`r50k_base`) 对英文 (`corpus1`) 和中文 (`corpus2`) 文本进行分词，并使用可视化工具进行对比。
    *   **结论：**
        *   **英文：** `tiktoken` 能有效分割出单词和词缀，压缩比较高；自实现的 Tokenizer 由于训练语料（中文手册）缺乏英文，倾向于将英文拆成单字母，压缩比较低。
        *   **中文：** `tiktoken` 对中文处理不佳，出现大量乱码和无效分割；自实现的 Tokenizer 由于在中文语料上训练，能识别出一些有意义的中文词语（如“博士”、“学位论文”），压缩比较 `tiktoken` 更高，但仍存在 `<unknown>` 标记，说明训练不足以覆盖所有必要的 UTF-8 字节组合。
        *   **通用：** `tiktoken` 词表更大更通用，处理未登录词能力更强；自实现 Tokenizer 的表现高度依赖于训练语料。

*   **回答概念性问题：**
    *   **工作：** 基于对 Tokenization 的理解，回答了关于 Unicode 操作、词表大小影响、LLM 处理特定任务（字符串反转、非英语、算术、代码）的局限性、特殊标记（`<|endoftext|>`）、未登录词（`SolidGoldMagikarp`）、数据格式（YAML vs JSON）以及端到端模型本质等问题。
    *   **结论：** Tokenization 机制深刻影响了 LLM 的能力和行为，解释了其在特定任务上表现不佳或出现特定现象的原因。例如，BPE 合并后的 token 粒度限制了模型进行细粒度操作（如字符级反转）的能力；训练语料的覆盖度和词表设计影响了对不同语言、数字和代码的处理效果。


# 1. BPE 算法原理：

Tokenization 面临两个挑战：
- 如果用全部可能的字符作为词表，词表过大。
- 如果不解决未登录词问题，会导致信息损失、模型表达能力受限、泛化能力受限等问题。

我们需要一种兼顾 **“表达能力”** 和 **“信息压缩率”** 的方法进行 tokenization。


一些 insight：
- 计算机中的字符可以通过编码以字节流的形式呈现。
- 字节的独特值有限：$2^8=256$。
- 理论上，在特定编码规则下，任意字符序列可以看作字节的排列组合。


基于上面的 insight，我们很自然的想到可以直接用 byte sequence 作为 tokenization 的结果，但是这样做的问题是：
- 基于单 byte 的编码方式 - ASCII，只能表达有限字符：英文字母、数字、控制符等等，难以适应多语言需求（比如中文）
- 基于多 byte 的编码方式 - 比如 Unicode+UTF-8，是可变编码，一个字符可能由一个或多个 byte 表示。虽然可以表示理论上所有语言的字符，但是在 “训练一个语言模型” 这个情景中，面临一些挑战：
    - **序列过长**：原始的 Transformer 是 $n^2$ 的，UTF-8 会使得序列进一步变长，显存占用和计算量增加。
    - **缺乏语义信息**：单个 byte 在 UTF-8 中可能不具有语义信息，不是一个完整的字符，如果让模型直接学习单独的 byte，可能输出 UTF-8 无法解码的序列。
    - **难以学习**：上方缺乏语义信息的问题会导致单纯的 UTF-8 byte sequence 破坏了人类语言的逻辑性和语义，如果模型直接学习这个 byte sequence，意味着它甚至要先学一遍 UTF-8 编码规则，才能接触到人类语义，非常有挑战。（Q：那为什么不用 Unicode 的 UTF-8 编码规则作为词表呢？A： BERT 词表 12w，Unicode 截止到今年大约 15w，其实没差多少，但是单纯的 Unicode 词表存在严重的长尾分布问题，所以模型事实上很难学到长尾字符相关的信息，我们应该为高频出现的字符/字符组合分配单独的 representation，从而使得模型可以从字符序列中获得更加密集的监督信息）
    - **没有充分利用统计信息**：Unicode 虽然有 15w 个字符，常用字符组合的数量远小于 15w \* 15w，所以充分利用词频信息可以大大压缩序列长度（很多编码系统的思想，比如 Huffman 编码，高频序列短编码，低频序列长编码）

    总的来说，我们可以先基于统计信息，提取高频 byte pattern，使得 tokenize 后的序列：
    - 语义化（让模型直接学习人类的语言逻辑）
    - 压缩率适中（适配 Transformer 的计算性质）
    - 监督信号密集（深度学习长久面临的问题）



# 2. BPE 算法实现 LLM tokenizer 的方式：

讲了上面的 motivation，真正的 BPE for LLM 算法就呼之欲出了：
1. 用 Unicode + UTF8 将字符序列转化为 byte 序列，以 256 unique byte 作为初始词表。
2. 用 byte pair 的频率作为当前词表下的 byte pattern，将最高频率的 pattern 加入词表。
3. 用新的词表对原始的 byte sequence 进行贪心匹配（尝试用最长的 byte pattern / character 进行匹配）。
4. 回到 2 进行新一轮的词表扩充，直到词表达到目标长度。

# 3. BPE 算法实现

我们首先实现一个基础的 BPE tokenizer，用最简单的代码实现 BPE，完全不考虑性能：

In [72]:
class BasicTokenizer:
    def __init__(self):
       self.init_vocab()

    def init_vocab(self, vocab: list[bytes] = None) -> None:
        """
        Initialize the vocabulary.
        """
        self.vocab = vocab if vocab else [bytes([i]) for i in range(256)] 

    def train(self, text: str, vocab_size: int) -> None:
        """
        Train the tokenizer using BPE algorithm.
        Params:
            text (str): string-type data used to run BPE.
            vocab_size (int): the size of final vocabulary.

        Return:
            None
        """
        byte_seqeunce = text.encode('utf-8')
        while len(self.vocab) < vocab_size:
            pattern_freq = self._get_byte_pattern_frequency(byte_seqeunce)
            if len(pattern_freq) == 0:
                break
            max_freq_pair = max(pattern_freq.items(), key=lambda x: x[1])
            pattern, freq = max_freq_pair
            self.vocab.append(pattern)
            # print(len(self.vocab))
    
    def encode(self, text: str) -> list[int]:
        """
        Encode the input string into a token list.
        Params:
            text (str): data to be tokenized.

        Return:
            ids (list): list of integer-type tokens.
        """
        byte_seqeunce = text.encode('utf-8')
        return [token[1] for token in self._pre_encode(byte_seqeunce)]

    def decode(self, ids: list[int]) -> str:
        """
        Decode a token list into a string.
        Params:
            ids (list): list of integer-type tokens.

        Return:
            text (str): string-type data.
        """
        byte_sequence = bytes()
        for idx in ids:
            byte_sequence += self.vocab[idx]
        decoded = byte_sequence.decode("utf-8", errors="replace")
        return decoded.replace('\ufffd', '<unknown>')


    def _get_byte_pattern_match(self, byte_seqeunce: bytes) -> tuple[bytes, int] | None:
        """
        Get the longest byte pattern match in the byte sequence.
        """
        for idx, pattern in enumerate(reversed(self.vocab)):
            if byte_seqeunce.startswith(pattern):
                return (pattern, len(self.vocab) - idx - 1)
        return None

    def _pre_encode(self, byte_seqeunce: bytes) -> list[tuple[bytes, int]]:
        """
        Pre-encode the input byte sequence into a list of byte pattern and its index.
        """
        tokens = []
        while byte_seqeunce:
            match = self._get_byte_pattern_match(byte_seqeunce)
            if match:
                tokens.append(match)
                byte_seqeunce = byte_seqeunce[len(match[0]):]
            else:
                raise ValueError(f"No match found for byte sequence: {byte_seqeunce}")
        return tokens

    def _get_byte_pattern_frequency(self, byte_seqeunce: bytes) -> list[tuple[tuple[int, int], int]]:
        """
        Get the frequency of the byte pattern in the byte sequence.
        """
        tokens = self._pre_encode(byte_seqeunce)
        if len(tokens) <= 1:
            return {}
        pairs = zip(tokens[:-1], tokens[1:])
        pattern_freq = {}
        for pair in pairs:
            pattern = pair[0][0] + pair[1][0]
            pattern_freq[pattern] = pattern_freq.get(pattern, 0) + 1
        return pattern_freq
    
    

这个基础的 BPE 实现显然是很慢的，因为它每次都会重建分词结果和频率表，查询匹配也是基于贪心和遍历的匹配方法，因此我们可以做一些简单的优化：
- 基于当前词表的最大 character 长度 + Hash Table 匹配最长前缀。
- 遍历当前 corpus 分割结果，合并 most frequent pair，而不是直接重建分割结果。
实现如下：

In [73]:
class FasterTokenizer:
    def __init__(self):
       self.init_vocab()

    def init_vocab(self, vocab: list[bytes] = None) -> None:
        """
        Initialize the vocabulary.
        """
        self.vocab = vocab if vocab else [bytes([i]) for i in range(256)]
        self.vocab_to_id = {v: i for i, v in enumerate(self.vocab)}
        self.max_len_vocab = max(len(v) for v in self.vocab)

    def train(self, text: str, vocab_size: int) -> None:
        """
        Train the tokenizer using BPE algorithm.
        Params:
            text (str): string-type data used to run BPE.
            vocab_size (int): the size of final vocabulary.

        Return:
            None
        """
        import time
        
        start_time = time.time()
        
        byte_seqeunce = text.encode('utf-8')
        ids = self._byte_sequence_to_token_ids(byte_seqeunce)
        if len(ids) <= 1:
            return

        pattern_freq = self._get_pattern_freq(ids)

        while len(self.vocab) < vocab_size:
            if len(pattern_freq) == 0:
                break
            max_freq_pair = max(pattern_freq.items(), key=lambda x: x[1])
            pattern, freq = max_freq_pair
            byte_pattern = self.vocab[pattern[0]] + self.vocab[pattern[1]]
            self.max_len_vocab = max(self.max_len_vocab, len(byte_pattern))
            self.vocab.append(byte_pattern)
            new_id = len(self.vocab) - 1
            self.vocab_to_id[byte_pattern] = new_id
            ids, pattern_freq = self._merge_tokens(ids, pattern_freq, pattern, new_id)
            # print(len(self.vocab))
        
        end_time = time.time()
        self.training_time = end_time - start_time
    
    def encode(self, text: str) -> list[int]:
        """
        Encode the input string into a token list.
        Params:
            text (str): data to be tokenized.

        Return:
            ids (list): list of integer-type tokens.
        """
        byte_seqeunce = text.encode('utf-8')
        return self._byte_sequence_to_token_ids(byte_seqeunce)

    def decode(self, ids: list[int]) -> str:
        """
        Decode a token list into a string.
        Params:
            ids (list): list of integer-type tokens.

        Return:
            text (str): string-type data.
        """
        byte_sequence = bytes()
        for idx in ids:
            byte_sequence += self.vocab[idx]
        decoded = byte_sequence.decode("utf-8", errors="replace")
        return decoded.replace('\ufffd', '<unknown>')


    def _get_byte_pattern_match(self, byte_seqeunce: bytes) -> tuple[bytes, int] | None:
        """
        Get the longest byte pattern match in the byte sequence.
        """
        for length in range(self.max_len_vocab, 0, -1):
            pattern = byte_seqeunce[:length]
            if pattern in self.vocab_to_id:
                return (pattern, self.vocab_to_id[pattern])
        return None

    def _byte_sequence_to_token_ids(self, byte_seqeunce: bytes) -> list[int]:
        """
        Pre-encode the input byte sequence into a list of byte pattern and its index.
        """
        ids = []
        while byte_seqeunce:
            match = self._get_byte_pattern_match(byte_seqeunce)
            if match:
                pattern, idx = match
                ids.append(idx)
                byte_seqeunce = byte_seqeunce[len(pattern):]
            else:
                raise ValueError(f"No match found for byte sequence: {byte_seqeunce}")
        return ids
    
    def _merge_tokens(self, ids: list[int], pattern_freq: dict[tuple[int, int], int], pattern: tuple[int, int], new_id: int) -> tuple[list[int], dict[tuple[int, int], int]]:
        """
        Merge the tokens in the list according to the pattern.
        """
        new_ids = []

        idx = 0
        while idx < len(ids):
            if idx + 1 < len(ids) and ids[idx] == pattern[0] and ids[idx + 1] == pattern[1]:
                new_ids.append(new_id)
                idx += 2
            else:
                new_ids.append(ids[idx])
                idx += 1

        return new_ids, self._get_pattern_freq(new_ids)
    
    def _get_pattern_freq(self, ids: list[int]) -> dict[tuple[int, int], int]:
        """
        Get the frequency of the pattern in the list.
        """
        pattern_freq = {}
        for pair in zip(ids[:-1], ids[1:]):
            pattern = (pair[0], pair[1])
            pattern_freq[pattern] = pattern_freq.get(pattern, 0) + 1
        return pattern_freq

现在我们可以来测试这两种 BPE 实现的性能和准确性：

In [74]:
corpus_file_path = 'manual.txt'

with open(corpus_file_path, 'r') as f:
    corpus = f.read()

def test_tokenizer(tokenizer, text: str, vocab_size: int, n_trials: int = 3):
    from time import time
    import numpy as np
    from termcolor import colored
    
    train_times = []
    encode_times = []
    decode_times = [] 
    
    for _ in range(n_trials):
        tokenizer.init_vocab()

        start_time = time()
        tokenizer.train(text, vocab_size)
        train_times.append(time() - start_time)
        
        start_time = time()
        ids = tokenizer.encode(text)
        encode_times.append(time() - start_time)
        
        start_time = time()
        decoded = tokenizer.decode(ids)
        decode_times.append(time() - start_time)
    
    train_mean, train_std = np.mean(train_times), np.std(train_times)
    encode_mean, encode_std = np.mean(encode_times), np.std(encode_times)
    decode_mean, decode_std = np.mean(decode_times), np.std(decode_times)

    
    print(colored(f"Training time: {train_mean:.3f}s ± {train_std:.3f}s", "cyan"))
    print(colored(f"Encoding time: {encode_mean:.3f}s ± {encode_std:.3f}s", "cyan")) 
    print(colored(f"Decoding time: {decode_mean:.3f}s ± {decode_std:.3f}s", "cyan"))
    print(colored(f"Accuracy (decoded == text): {decoded == text}", "green" if decoded == text else "red"))
    
    return ids, train_mean, encode_mean, decode_mean

In [85]:
basic_tokenizer = BasicTokenizer()
faster_tokenizer = FasterTokenizer()

vocab_size = 1024
corpus_size = [100, 200, 400, 800, 1600]

for size in corpus_size:
    from termcolor import colored
    
    print(colored(f"========== Corpus size: {size} ==========", "yellow", attrs=["bold"]))
    
    print(colored("Basic tokenizer:", "blue", attrs=["bold"]))
    ids_basic, train_basic, encode_basic, decode_basic = test_tokenizer(basic_tokenizer, corpus[:size], vocab_size)
    print()
    
    print(colored("Faster tokenizer:", "blue", attrs=["bold"]))
    ids_faster, train_faster, encode_faster, decode_faster = test_tokenizer(faster_tokenizer, corpus[:size], vocab_size)
    print()
    
    train_speedup = train_basic / train_faster if train_faster > 0 else float('inf')
    encode_speedup = encode_basic / encode_faster if encode_faster > 0 else float('inf')
    decode_speedup = decode_basic / decode_faster if decode_faster > 0 else float('inf')
    
    print(colored("Performance comparison:", "magenta", attrs=["bold"]))
    print(colored(f"Training speedup: {train_speedup:.2f}x", "cyan"))
    print(colored(f"Encoding speedup: {encode_speedup:.2f}x", "cyan"))
    print(colored(f"Decoding speedup: {decode_speedup:.2f}x", "cyan"))
    print()
    
    # Check if tokenization results match
    tokens_match = ids_basic == ids_faster
    print(colored("Tokenization comparison:", "magenta", attrs=["bold"]))
    print(colored(f"Basic tokenizer [{len(ids_basic)}] == Faster tokenizer [{len(ids_faster)}]: {tokens_match}", 
          "green" if tokens_match else "red"))
    print("\n\n")


========== Corpus size: 100 ==========
Basic tokenizer:
Training time: 0.065s ± 0.015s
Encoding time: 0.000s ± 0.000s
Decoding time: 0.000s ± 0.000s
Accuracy (decoded == text): True

Faster tokenizer:
Training time: 0.002s ± 0.000s
Encoding time: 0.000s ± 0.000s
Decoding time: 0.000s ± 0.000s
Accuracy (decoded == text): True

Performance comparison:
Training speedup: 36.59x
Encoding speedup: 3.29x
Decoding speedup: 9.70x

Tokenization comparison:
Basic tokenizer [1] == Faster tokenizer [1]: True



========== Corpus size: 200 ==========
Basic tokenizer:
Training time: 0.352s ± 0.002s
Encoding time: 0.000s ± 0.000s
Decoding time: 0.000s ± 0.000s
Accuracy (decoded == text): True

Faster tokenizer:
Training time: 0.010s ± 0.000s
Encoding time: 0.000s ± 0.000s
Decoding time: 0.000s ± 0.000s
Accuracy (decoded == text): True

Performance comparison:
Training speedup: 34.32x
Encoding speedup: 1.13x
Decoding speedup: 1.76x

Tokenization comparison:
Basic tokenizer [1] == Faster tokenizer [1]: 

从上面的性能对比可以看到：
- 训练加速比随着语料规模增大逐渐增加，符合我们对复杂度的估计。
- 当词表大小大于语料规模时，整个语料会被单独的 token 所表示，此时属于过拟合，后端的模型无法学习到任何有效信息。
- 两个 tokenizer 编码后的 token 序列不同，token 序列长度不同，词表大小也不同，压缩比不同，这是因为 token 匹配和合并的过程不是确定性的，因此不同的 tokenizer 可能产生差别。

In [76]:
import base64

def save_vocab(tokenizer, filepath: str = 'vocab.txt') -> None:
    """
    Save the vocabulary list to a file.
    Each byte token is base64 encoded and written to a new line.
    """
    vocab = tokenizer.vocab
    with open(filepath, 'w', encoding='utf-8') as f:
        token_str = []
        for token_bytes in vocab:
            token_str.append(base64.b64encode(token_bytes).decode('utf-8')+ '\n')
        f.writelines(token_str)
    

def load_vocab(tokenizer, filepath: str = 'vocab.txt') -> None:
    """
    Load the vocabulary list from a file.
    Assumes the file contains base64 encoded tokens, one per line.
    """
    loaded_vocab = []
    with open(filepath, 'r', encoding='utf-8') as f:
        for line in f:
            line = line.strip()
            if line:
                token_bytes = base64.b64decode(line.encode('utf-8'))
                loaded_vocab.append(token_bytes)

    tokenizer.init_vocab(vocab=loaded_vocab)

In [77]:
import os

final_tokenizer = FasterTokenizer()

if not os.path.exists('vocab.txt'):
    final_tokenizer.init_vocab()
    final_tokenizer.train(corpus, 1024)
    save_vocab(final_tokenizer, 'vocab.txt')
else:
    load_vocab(final_tokenizer, 'vocab.txt')

In [78]:
ids = final_tokenizer.encode(corpus)
decoded = final_tokenizer.decode(ids)
assert decoded == corpus

In [79]:
import tiktoken

openai_tokenizer = tiktoken.get_encoding('r50k_base')

In [80]:
corpus1 = 'Originated as the Imperial University of Peking in 1898, Peking University was China’s first national comprehensive university and the supreme education authority at the time. Since the founding of the People’s Republic of China in 1949, it has developed into a comprehensive university with fundamental education and research in both humanities and science. The reform and opening-up of China in 1978 has ushered in a new era for the University unseen in history. And its merger with Beijing Medical University in 2000 has geared itself up for all-round and vibrant growth in such fields as science, engineering, medicine, agriculture, humanities and social sciences. Supported by the “211 Project” and the “985 Project”, the University has made remarkable achievements, such as optimizing disciplines, cultivating talents, recruiting high-caliber teachers, as well as teaching and scientific research, which paves the way for a world-class university.'
corpus2 = '博士学位论文应当表明作者具有独立从事科学研究工作的能力，并在科学或专门技术上做出创造性的成果。博士学位论文或摘要，应当在答辩前三个月印送有关单位，并经同行评议。学位授予单位应当聘请两位与论文有关学科的专家评阅论文，其中一位应当是外单位的专家。评阅人应当对论文写详细的学术评语，供论文答辩委员会参考。'

In [81]:
def visualize_tokens(tokens: list[str], separator: str = ' '):
    """
    Visualize a list of tokens with random colors using termcolor.
    
    Args:
        tokens: List of tokens (strings) to visualize
        separator: String to use between tokens (default is a space)
        
    Returns:
        A colored string representation of the tokens
    """
    import random
    from termcolor import colored

    colors = ['red', 'green', 'yellow', 'blue', 'magenta', 'cyan', 'grey', 'light_red', 'light_green']
    
    colored_tokens = []
    used_styles = {}
    last_color = None
    
    for token in tokens:
        if token in used_styles:
            color = used_styles[token]
        else:
            available_colors = [c for c in colors if c != last_color]
            if not available_colors:
                available_colors = colors
            
            color = random.choice(available_colors)
            used_styles[token] = color
        
        colored_tokens.append(colored(token, color))
        last_color = color
    
    result = separator.join(colored_tokens)
    return result

In [82]:
import tiktoken
from termcolor import colored

def visualize_tiktoken(text, encoding_name='r50k_base'):
    """Visualize tokenization with tiktoken"""
    encoder = tiktoken.get_encoding(encoding_name)
    tokens = encoder.encode(text)
    
    token_texts = []
    for token in tokens:
        token_text = encoder.decode([token])
        token_texts.append(token_text)
    
    print(f"Tiktoken Tokenization ({len(token_texts)} tokens):")
    print(visualize_tokens(token_texts))
    
    return token_texts

def visualize_custom_tokenizer(text, tokenizer):
    """Visualize tokenization with custom tokenizer"""
    token_ids = tokenizer.encode(text)
    
    token_texts = []
    for token_id in token_ids:
        token_text = tokenizer.decode([token_id])
        token_texts.append(token_text)
    
    print(f"Custom Tokenization ({len(token_texts)} tokens):")
    print(visualize_tokens(token_texts))
    
    return token_texts

def compare_tokenizers(text, custom_tokenizer):
    """Compare tiktoken and custom tokenizer on the same text"""
    print("=" * 80)
    print(f"Text: {text[:80]}{'...' if len(text) > 80 else ''}")
    print("=" * 80)
    
    print("\n--- TIKTOKEN ---")
    tiktoken_tokens = visualize_tiktoken(text)
    
    print("\n--- CUSTOM TOKENIZER ---")
    custom_tokens = visualize_custom_tokenizer(text, custom_tokenizer)
    
    # Calculate compression ratio
    tiktoken_ratio = len(text) / len(tiktoken_tokens)
    custom_ratio = len(text) / len(custom_tokens)
    
    print("\n--- COMPARISON ---")
    print(f"Text length: {len(text)} characters")
    print(f"Tiktoken: {len(tiktoken_tokens)} tokens (compression ratio: {tiktoken_ratio:.2f})")
    print(f"Custom: {len(custom_tokens)} tokens (compression ratio: {custom_ratio:.2f})")
    print("=" * 80)
    print("\n")

# Run comparison for both corpus texts
compare_tokenizers(corpus1, final_tokenizer)
compare_tokenizers(corpus2, final_tokenizer)

Text: Originated as the Imperial University of Peking in 1898, Peking University was C...

--- TIKTOKEN ---
Tiktoken Tokenization (185 tokens):
Orig inated  as  the  Imperial  University  of  P eking  in  1898 ,  P eking  University  was  China � � s  first  national  comprehensive  university  and  the  supreme  education  authority  at  the  time .  Since  the  founding  of  the  People � � s  Republic  of  China  in  1949 ,  it  has  developed  into  a  comprehensive  university  with  fundamental  education  and  research  in  both  humanities  and  science .  The  reform  and  opening - up  of  China  in  1978  has  ushered  in  a  new  era  for  the  University  unseen  in  history .  And  its  merger  with  Beijing  Medical  University  in  2000  has  geared  itself  up  for  all - round  and  vibrant  growth  in  such  fields  as  science ,  engineering ,  medicine ,  agriculture ,  humanities  and  social  sciences .  Supported  by  the  � � 211  Project � �  and  the  � � 985

结果非常有趣，我们逐项分析：
- 对于英文语料：GPT2 tokenizer 准确的分割出了单词，同时出现了词缀分割（比如 Orig-inated），这是 BPE 预期的结果：通过学习语料中的词频，自动发掘常见的模式（比如固定的字母组合=单词）。而我们实现的 tokenizer 把所有单词分割为单独的字母，这是因为我们的训练语料中没有英文，所以词表中仅有英文字母，却没有单词。
- 对于中文语料：GPT2  tokenizer 基本上没有输出任何有意义的信息，其中大部分还是乱码（可能由于编码系统的差异？）。而我们实现的 tokenizer，由于在北大学生手册上经过学习，所以捕捉到了一些有效词汇（比如 “博士”，“学位授予”， “学位论文”）。
- 纵观两种语料上的分割结果：GPT2 tokenizer 基本上没有 `<unknown>`，但是我们实现的 tokenizer 出现了大量的 `<unknown>`，这是因为在 Unicode UTF-8 中需要多个 byte 才能表示有效中文字符，由于我们训练语料过小，所以还没有充分习得有效的 UTF-8 表示（当我们词表大小超过 15w，也就是 Unicode 的大小， 才有希望完全消除 unknown 的问题）。

## 1.3 回答问题

**Q1：Python中使用什么函数查看字符的Unicode，什么函数将Unicode转换成字符？并使用它们查看“北”“大”的Unicode，查看Unicode为22823、27169、22411对应的字符。**

In [83]:
print(ord('北'))
print(ord('大'))
print(chr(22823))
print(chr(27169))
print(chr(22411))


21271
22823
大
模
型


**Q2：Tokenizer的vocab size大和小分别有什么好处和坏处？**

A：答：
- 大的好处：可以尽可能捕捉词频对应的语义信息（比如学会语料中的所有有效字符组合/单词）；语料压缩率更高。
- 大的坏处：LLM 学习困难（太多罕见词，长尾问题），在语料上过拟合，缺乏泛化性，分词结果粒度太粗。
- 小的好处：训练快，泛化性好（避免在中文上学习太多的字符组合，而是着重理解每一个字符的语义以及捕捉字符间关系，如果提前把字符组合起来，会导致模型无法学习单独语素的含义）
- 小的坏处：压缩率低，词汇被拆散从而破坏予以信息。


**Q3：为什么 LLM 不能处理非常简单的字符串操作任务，比如反转字符串？**

A：在 BPE tokenizer 训练的过程中，字母自发组合为单词，又由于 tokenizer encode 的过程中，词汇通过最长贪心匹配来分割语料，导致模型对文本操作的最小单元受限于 tokenizer 训练的结果，比如一个完整的单词，而无法细粒度的操作单词中的字母。

**Q4：为什么 LLM 在非英语语言（例如日语）上表现较差？**

A：有几个可能的原因：
- 英语语料之间有天然的分隔符（空格），使得英语的分割结果比较符合语言本身的语义（比如 BPE 可以学到真正的单词），但是对于中日韩这种语素文字系统，没有天然的词汇分隔符，所以 BPE 较难学到有效的词汇表示。
- 中日韩语存在歧义多义问题，一句话可以用多种不同方法分割。
- 中日韩语属于孤立语/黏着语，语素本身具有一定语义，语素的组合会产生更加复杂多变的语义，难以学习。
- 在 Unicode 系统中一个非英文字符需要多个 byte 来表示，相比于英文语料可以直接用初始的 256 词表组合，中日韩语的语料需要更长时间才能逐步发现有语义的 byte 组合。

**Q5：为什么 LLM 在简单算术问题上表现不好？**

A：从 tokenization 的角度看，有几个可能的原因：
- 每一个独立的数字都可以用一个 byte 表示，因此绝大多数数字在 LLM 看来不是一个 token，而是多个 token 的组合，因此 LLM 不但需要理解数字的含义（相对大小），也要理解进制（十进制和十六进制等）、阅读方向（从右到左个十百千万...，与一般语言不同）、格式（英语书写系统中普遍采用 , 分隔千位）等等数学概念和书写系统。从这个角度看，即使是简单的算术问题也是高层次语义任务，需要模型有足够好的底层次数学符号和概念理解。
- 训练预料中数字出现的频率会极大的影响分词结果中数字的表达形式，但这是不对的，因为我们对数学，有别于语言，有一套通用的专门的规则，这部分不应该由 BPE 通过词频学习，而是应该直接在 pre-tokenize 阶段显式的表达。

**Q6：为什么 GPT-2 在编写 Python 代码时遇到比预期更多的困难？**

A：从 tokenization 角度，可能的原因：
- 代码是一个基于规则的符号系统，有严格的约定，更适合基于语法规则直接分词而不是让 BPE 学习（毕竟学习的终极目标就是这套规则，所以没必要本末倒置）
- 代码中存在基于规则的符号系统（代码，数字）和非规则的语言系统（字符串），如何处理两种系统的边界，兼容两种系统的符号，依靠单纯的 BPE 是不能很好解决的。

**Q7：为什么 LLM 遇到字符串 “<|endoftext|>” 时会突然中断？**

A：这是训练 LLM 时约定的文本结束符号。

**Q8：为什么当问 LLM 关于 “SolidGoldMagikarp” 的问题时 LLM 会崩溃？**

A：因为这是一个没有意义的字符串，在训练语料中没有出现过，所以 LLM 无法理解这个字符串的含义，从而导致崩溃（OOD 问题）。

**Q9：为什么在使用 LLM 时应该更倾向于使用 YAML 而不是 JSON？**

A：因为 YAML 的语法更符合人类的阅读习惯，更容易被人类理解，而 JSON 的语法则包含更多的 “规则”（比如括号匹配）， 适合用代码解析而不是直接阅读。

**Q10：为什么 LLM 实际上不是端到端的语言建模？**

A：因为这个链条事实上是 text > token > embedding > LLM > output logits > embedding > token > text，所以部分的表征学习由 tokenizer 和 embedding 负责，模型没有直接学习语言字符系统。从这个角度来看，现在的 Audio2Autio 多模态模型（比如 Qwen-Omni）反而更加接近语言的本质。